In [1]:
% cd /home/mayu-ot/Documents/iparaphrasing-release

/home/mayu-ot/Documents/iparaphrasing-release


In [2]:
import pandas as pd
import numpy as np
import sklearn.metrics
from codes.func.util.html_template import *
from IPython.display import display_html

In [3]:
def split_data_word_count(df):
    # split validation set
    mult_idx = []
    single_idx = []

    for i, row in df.iterrows():
        phr1 = row['phrase1']
        phr2 = row['phrase2']

        if ('+' in phr1) and ('+' in phr2):
            mult_idx.append(i)
        elif ('+' not in phr1) and ('+' not in phr2):
            single_idx.append(i)
    
    return single_idx, mult_idx

def get_best_thresh(df, index=None):
    # eval on single word examples
    items = df if index is None else df.iloc[index]
    score = items.score
    ytrue = items.ytrue

    prec, rec, thresh = sklearn.metrics.precision_recall_curve(y_true=ytrue, probas_pred=score)
    f1 = 2 * (prec * rec) / (prec + rec)
    f1[np.isnan(f1)] = 0.
    best_i = f1.argmax()
    best_thresh = thresh[best_i]
    
    return best_thresh

def compute_scores(df, thresh, index=None):
    # eval_on multple word examples
    items = df if index is None else df.iloc[index]
    score = items.score
    ytrue = items.ytrue

    prec = sklearn.metrics.precision_score(y_true=ytrue, y_pred=score>thresh)
    rec = sklearn.metrics.recall_score(y_true=ytrue, y_pred=score>thresh)
    f1 = sklearn.metrics.f1_score(y_true=ytrue, y_pred=score>thresh)

    return {'precision': prec, 'recall': rec, 'f1': f1}
    

def get_results(val_df, test_df):
    val_single_idx, val_mult_idx = split_data_word_count(val_df)
    test_single_idx, test_mult_idx = split_data_word_count(test_df)
    
    best_thresh = get_best_thresh(val_df, index=val_single_idx)
    single_res = compute_scores(test_df, best_thresh, index=test_single_idx)
    
    best_thresh = get_best_thresh(val_df, index=val_mult_idx)
    mult_res = compute_scores(test_df, best_thresh, index=test_mult_idx)
    
    best_thresh = get_best_thresh(val_df)
    all_res = compute_scores(test_df, best_thresh)
    
    return single_res, mult_res, all_res

In [4]:
def get_model_ensemble_df(model_dir1, model_dir2):
    # compute best threshold on validation set
    ens_df = []
    
    for split in ['val', 'test']:
        res1 = pd.read_csv('%s/res_%s.csv'%(model_dir1, split))
        res2 = pd.read_csv('%s/res_%s.csv'%(model_dir2, split))

        p_score = res1.score.values
        i_score = res2.score.values

        ens_score = (p_score + i_score) * .5
        ens_res = res1.copy()
        ens_res.score = ens_score
        
        ens_df.append(ens_res)
    
    return ens_df[0], ens_df[1]

In [5]:
methods = {
    'PL': 'pl',
    'TP': 'tp',
    'WEA': 'wea',
    'FV': 'fv+pca',
    'FV+CCA': 'fv+cca',
    'SNN (WEA)': 'avr-None',
    'SNN+image (WEA)' : 'avr-vgg',
    'Ensemble (WEA)': ('avr-None', 'avr-vgg'),
    'SNN (FV)': 'fv+pca-None',
    'SNN+image (FV)': 'fv+pca-vgg',
    'Ensemble (FV)': ('fv+pca-None', 'fv+pca-vgg'),
    'SNN (FV+CCA)': 'fv+cca-None',
    'SNN+image (FV+CCA)': 'fv+cca-vgg',
    'Ensemble (FV+CCA)': ('fv+cca-None', 'fv+cca-vgg'),
    }

output_html = head
for m_name, items in methods.items():

    if m_name[:8] == 'Ensemble':
        val_df, test_df = get_model_ensemble_df('models/%s/' % items[0], 'models/%s/' % items[1])
    else:
        val_df = pd.read_csv('models/%s/' % items+'res_val.csv')
        test_df = pd.read_csv('models/%s/' % items + 'res_test.csv')
    
    single_res, mult_res, all_res = get_results(val_df, test_df)
    
    res_vals = [m_name]
    for met in ['precision', 'recall', 'f1']:
        for res in [all_res, single_res, mult_res]:
            res_vals.append(res[met] * 100)
    
    output_html += row % tuple(res_vals)

output_html += bottom

display_html(output_html, raw=True)

/home/mayu-ot/miniconda2/envs/py36chainer/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


Precision 
 Recall 
 F1 
 
 
 
 all 
 single 
 multi 
 all 
 single 
 multi 
 all 
 single 
 multi 
 

 
 PL 
 59.32 
 51.53 
 62.86 
 63.12 
 47.99 
 74.14 
 61.16 
 49.70 
 68.04 
 

 
 TP 
 66.23 
 63.20 
 82.17 
 64.20 
 66.10 
 56.31 
 65.20 
 64.62 
 66.83 
 

 
 WEA 
 61.51 
 46.11 
 62.84 
 71.29 
 67.93 
 78.47 
 66.04 
 54.93 
 69.79 
 

 
 FV 
 63.79 
 40.84 
 67.51 
 60.87 
 35.41 
 77.03 
 62.30 
 37.94 
 71.96 
 

 
 FV+CCA 
 64.79 
 55.79 
 68.24 
 82.20 
 75.83 
 84.98 
 72.46 
 64.28 
 75.69 
 

 
 SNN (WEA) 
 77.86 
 83.66 
 74.50 
 84.58 
 75.16 
 88.96 
 81.08 
 79.18 
 81.09 
 

 
 SNN+image (WEA) 
 79.47 
 81.01 
 77.26 
 84.56 
 79.35 
 87.06 
 81.94 
 80.17 
 81.86 
 

 
 Ensemble (WEA) 
 80.65 
 78.68 
 77.38 
 84.79 
 83.14 
 88.85 
 82.67 
 80.85 
 82.72 
 

 
 SNN (FV) 
 64.21 
 45.92 
 66.40 
 65.93 
 50.89 
 76.51 
 65.06 
 48.28 
 71.10 
 

 
 SNN+image (FV) 
 63.49 
 52.62 
 66.86 
 68.20 
 55.62 
 78.01 
 65.76 
 54.08 
 72.01 
 

 
 Ensemble (FV) 
 65.48 
 54.87 
 70.51 
 71.43 
 56.24 
 76.54 
 68.33 
 55.55 
 73.40 
 

 
 SNN (FV+CCA) 
 83.11 
 85.19 
 77.44 
 82.13 
 79.30 
 87.69 
 82.62 
 82.14 
 82.25 
 

 
 SNN+image (FV+CCA) 
 82.51 
 84.52 
 80.28 
 84.19 
 81.85 
 86.82 
 83.34 
 83.16 
 83.43 
 

 
 Ensemble (FV+CCA) 
 82.71 
 84.10 
 80.91 
 85.67 
 83.50 
 87.06 
 84.16 
 83.80 
 83.87